In [ ]:
import base64
import numpy as np
import pandas as pd
import psycopg2
import re
import spacy
import nltk
import numexpr
nltk.download('punkt')

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from wordcloud import WordCloud, STOPWORDS
from pprint import pprint
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
%matplotlib inline

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

!pip install -U pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
conn = psycopg2.connect(
    host="codd04.research.northwestern.edu",
    port = "5433",
    database="postgres",
    user="cpdbstudent",
    password="DataSci4AI")

In [ ]:
cursor = conn.cursor()

In [ ]:
data_allegation_summary = "select \
      da.crid, da.cr_text, doa.disciplined \
    from \
      (select crid, cr_text from data_allegation where cr_text is not null) as da \
        left join (select allegation_id, disciplined from data_officerallegation group by 1,2) as doa \
          on da.crid = doa.allegation_id \
"

In [ ]:
cursor.execute(data_allegation_summary)
summaries = cursor.fetchall()
print("shape is: " + str(len(summaries))) # 186983


df_summaries = pd.DataFrame(summaries)
colnames = [desc[0] for desc in cursor.description]
df_summaries.columns = colnames
print(df_summaries.shape)

shape is: 11303
(11303, 3)


In [ ]:
df_summaries.head

<bound method NDFrame.head of           crid                                            cr_text disciplined
0      1062172  Initial / Intake Allegation 1: The complainant...       False
1      1054276  Initial / Intake Allegation 1: The reporting p...       False
2      1049008  Initial / Intake Allegation 1: The reporting p...       False
3      1056461  Initial / Intake Allegation 1:  The reporting ...       False
4      1049161  Initial / Intake Allegation 1: The reporting p...       False
...        ...                                                ...         ...
11298  1053289  Initial / Intake Allegation 1:  The reporting ...        None
11299  1054371  Initial / Intake Allegation 1:  The reporting ...        None
11300  1053798  Initial / Intake Allegation 1:  The complainan...        None
11301  1059891  Initial / Intake Allegation 4: It is reported ...        None
11302  1049365  Initial / Intake Allegation 1:  The reporting ...        None

[11303 rows x 3 columns]>

In [ ]:
df_summaries.disciplined.value_counts()

False    2875
True      197
Name: disciplined, dtype: int64

In [ ]:
df_summaries_disciplined = df_summaries[df_summaries.disciplined == True]

df_summaries_disciplined.shape

(197, 3)

In [ ]:
df_summaries_notdisciplined = df_summaries[df_summaries.disciplined != True]

df_summaries_notdisciplined.shape

(11106, 3)

In [ ]:
data_discplined = df_summaries_disciplined['cr_text'].values.tolist()

data_discplined = [re.sub('\n+', ' ', sent) for sent in data_discplined]

In [ ]:
data_notdiscplined = df_summaries_notdisciplined['cr_text'].values.tolist()

data_notdiscplined = [re.sub('\n+', ' ', sent) for sent in data_notdiscplined]

In [ ]:
#Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

disciplined_words = list(sent_to_words(data_discplined))
notdisciplined_words = list(sent_to_words(data_notdiscplined))

pprint(disciplined_words[:1])
pprint(notdisciplined_words[:1])

[['initial',
  'intake',
  'allegation',
  'it',
  'is',
  'reported',
  'that',
  'the',
  'accused',
  'officers',
  'submitted',
  'false',
  'police',
  'report',
  'in',
  'that',
  'they',
  'identified',
  'officers',
  'who',
  'did',
  'not',
  'participate',
  'in',
  'an',
  'arrest',
  'and',
  'identified',
  'those',
  'officers',
  'as',
  'first',
  'arresting',
  'officer',
  'and',
  'second',
  'arresting',
  'officer',
  'it',
  'is',
  'reported',
  'that',
  'the',
  'accused',
  'officers',
  'submitted',
  'false',
  'police',
  'report',
  'in',
  'that',
  'they',
  'identified',
  'officers',
  'who',
  'did',
  'not',
  'participate',
  'in',
  'an',
  'arrest',
  'and',
  'identified',
  'those',
  'officers',
  'as',
  'first',
  'arresting',
  'officer',
  'and',
  'second',
  'arresting',
  'officer',
  'initial',
  'intake',
  'allegation',
  'it',
  'is',
  'reported',
  'that',
  'the',
  'accused',
  'officers',
  'submitted',
  'false',
  'police',


In [ ]:
# Build the bigram and trigram models
bigram_disc = gensim.models.Phrases(disciplined_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_disc = gensim.models.Phrases(bigram_disc[disciplined_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_disc_mod = gensim.models.phrases.Phraser(bigram_disc)
trigram_disc_mod = gensim.models.phrases.Phraser(trigram_disc)

# See trigram example
print(trigram_disc_mod[bigram_disc_mod[disciplined_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['initial', 'intake', 'allegation', 'it', 'is', 'reported', 'that', 'the', 'accused', 'officers', 'submitted', 'false', 'police', 'report', 'in', 'that', 'they', 'identified', 'officers', 'who', 'did', 'not', 'participate', 'in', 'an', 'arrest', 'and', 'identified', 'those', 'officers', 'as', 'first', 'arresting', 'officer', 'and', 'second', 'arresting', 'officer', 'it', 'is', 'reported', 'that', 'the', 'accused', 'officers', 'submitted', 'false', 'police', 'report', 'in', 'that', 'they', 'identified', 'officers', 'who', 'did', 'not', 'participate', 'in', 'an', 'arrest', 'and', 'identified', 'those', 'officers', 'as', 'first', 'arresting', 'officer', 'and', 'second', 'arresting', 'officer', 'initial', 'intake', 'allegation', 'it', 'is', 'reported', 'that', 'the', 'accused', 'officers', 'submitted', 'false', 'police', 'report', 'in', 'that', 'they', 'identified', 'officers', 'who', 'did', 'not', 'participate', 'in', 'an', 'arrest', 'and', 'identified', 'those', 'officers', 'as', 'first'

In [ ]:
# Build the bigram and trigram models
bigram_notdisc = gensim.models.Phrases(disciplined_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_notdisc = gensim.models.Phrases(bigram_notdisc[disciplined_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_notdisc_mod = gensim.models.phrases.Phraser(bigram_notdisc)
trigram_notdisc_mod = gensim.models.phrases.Phraser(trigram_notdisc)

# See trigram example
print(trigram_notdisc_mod[bigram_notdisc_mod[disciplined_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['initial', 'intake', 'allegation', 'it', 'is', 'reported', 'that', 'the', 'accused', 'officers', 'submitted', 'false', 'police', 'report', 'in', 'that', 'they', 'identified', 'officers', 'who', 'did', 'not', 'participate', 'in', 'an', 'arrest', 'and', 'identified', 'those', 'officers', 'as', 'first', 'arresting', 'officer', 'and', 'second', 'arresting', 'officer', 'it', 'is', 'reported', 'that', 'the', 'accused', 'officers', 'submitted', 'false', 'police', 'report', 'in', 'that', 'they', 'identified', 'officers', 'who', 'did', 'not', 'participate', 'in', 'an', 'arrest', 'and', 'identified', 'those', 'officers', 'as', 'first', 'arresting', 'officer', 'and', 'second', 'arresting', 'officer', 'initial', 'intake', 'allegation', 'it', 'is', 'reported', 'that', 'the', 'accused', 'officers', 'submitted', 'false', 'police', 'report', 'in', 'that', 'they', 'identified', 'officers', 'who', 'did', 'not', 'participate', 'in', 'an', 'arrest', 'and', 'identified', 'those', 'officers', 'as', 'first'

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, trigram_mod, bigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = stopwords.words('english')
data_disc_nostops = remove_stopwords(disciplined_words)
data_notdisc_nostops = remove_stopwords(notdisciplined_words)

# Form Bigrams
data_disc_bigrams = make_bigrams(data_disc_nostops, bigram_disc)
data_notdisc_bigrams = make_bigrams(data_notdisc_nostops, bigram_notdisc)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_disc_lemmatized = lemmatization(data_disc_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_notdisc_lemmatized = lemmatization(data_notdisc_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

pprint(data_disc_lemmatized[:1])
pprint(data_notdisc_lemmatized[:1])

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: Sele

[['initial',
  'intake',
  'allegation',
  'report',
  'accuse',
  'officer',
  'submit',
  'false',
  'police',
  'report',
  'identify',
  'officer',
  'participate',
  'arrest',
  'identify',
  'officer',
  'first',
  'arrest',
  'officer',
  'second',
  'arrest',
  'officer',
  'report',
  'accuse',
  'officer',
  'submit',
  'false',
  'police',
  'report',
  'identify',
  'officer',
  'participate',
  'arrest',
  'identify',
  'officer',
  'first',
  'arrest',
  'officer',
  'second',
  'arrest',
  'officer',
  'initial',
  'intake',
  'allegation',
  'report',
  'accuse',
  'officer',
  'submit',
  'false',
  'police',
  'report',
  'identify',
  'officer',
  'participate',
  'arrest',
  'identify',
  'officer',
  'first',
  'arrest',
  'officer',
  'second',
  'arrest',
  'officer',
  'report',
  'accuse',
  'officer',
  'submit',
  'false',
  'police',
  'report',
  'identify',
  'officer',
  'participate',
  'arrest',
  'identify',
  'officer',
  'first',
  'arrest',
  'offic

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use sele

In [ ]:
#The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.
# Create Dictionary
id2word_disc = corpora.Dictionary(data_disc_lemmatized)
id2word_notdisc = corpora.Dictionary(data_notdisc_lemmatized)

# Create Corpus
texts_disc = data_disc_lemmatized
texts_notdisc = data_notdisc_lemmatized

# Term Document Frequency
corpus_disc = [id2word_disc.doc2bow(text) for text in texts_disc]
corpus_notdisc = [id2word_notdisc.doc2bow(text) for text in texts_notdisc]

# View
print(corpus_disc[:1])
print(corpus_notdisc[:1])

[[(0, 8), (1, 4), (2, 3), (3, 2), (4, 1), (5, 5), (6, 13), (7, 3), (8, 1), (9, 4), (10, 1), (11, 1), (12, 1), (13, 4), (14, 4), (15, 5), (16, 8), (17, 4), (18, 1), (19, 2), (20, 2), (21, 2), (22, 23), (23, 3), (24, 4), (25, 4), (26, 8), (27, 1), (28, 14), (29, 1), (30, 4), (31, 4), (32, 4), (33, 1), (34, 10)]]
[[(0, 2), (1, 2), (2, 2), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 2), (9, 1), (10, 2), (11, 2), (12, 2)]]


In [ ]:
[[(id2word_disc[id], freq) for id, freq in cp] for cp in corpus_disc[:1]]

[[(id2word_notdisc[id], freq) for id, freq in cp] for cp in corpus_notdisc[:1]]

[[('accuse', 2),
  ('allegation', 2),
  ('allege', 2),
  ('complainant', 2),
  ('enter', 1),
  ('feel', 2),
  ('find', 1),
  ('initial', 2),
  ('intake', 2),
  ('none', 1),
  ('officer', 2),
  ('profile', 2),
  ('racially', 2)]]

In [ ]:
#you need to provide the number of topics as well.
# Build LDA model
lda_model_disc = gensim.models.ldamodel.LdaModel(corpus=corpus_disc,
                                           id2word=id2word_disc,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
#you need to provide the number of topics as well.
# Build LDA model
lda_model_notdisc = gensim.models.ldamodel.LdaModel(corpus=corpus_notdisc,
                                           id2word=id2word_notdisc,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [ ]:
!pip install pandas --upgrade

In [ ]:
pyLDAvis.enable_notebook()
vis_disc = gensimvis.prepare(lda_model_disc, corpus_disc, id2word_disc)
vis_disc

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.133607 -0.083775       1        1  36.849470
3     -0.275466 -0.112074       2        1  35.378599
1      0.134217 -0.020620       3        1  12.091531
2      0.058762 -0.053207       4        1   9.519110
0     -0.051119  0.269676       5        1   6.161289, topic_info=            Term        Freq       Total Category  logprob  loglift
8    complainant  352.000000  352.000000  Default  30.0000  30.0000
22       officer  328.000000  328.000000  Default  29.0000  29.0000
2         allege  556.000000  556.000000  Default  28.0000  28.0000
28        report  482.000000  482.000000  Default  27.0000  27.0000
15          hour  379.000000  379.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
394       assign    5.733728   20.621596   Topic5  -4.9164   1.5069
168          use    5.208712   16.353857   Topic5  -5.0125   1.6428
0         accuse   11.297881  650.863861   Topic5  -4.2382  -1.2668
34          unit    5.452702  111.753034   Topic5  -4.9667  -0.2333
251          say    4.292025   19.241127   Topic5  -5.2060   1.2866

[282 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
354       1  0.905565      ability
591       3  0.955248       access
0         1  0.585376       accuse
0         2  0.213562       accuse
0         3  0.129059       accuse
...     ...       ...          ...
132       1  0.061069         work
132       2  0.916042         work
132       4  0.017448         work
132       5  0.017448         work
601       3  0.955248  workstation

[373 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 3, 1])

In [ ]:
pyLDAvis.enable_notebook()
vis_notdisc = gensimvis.prepare(lda_model_notdisc, corpus_notdisc, id2word_notdisc)
vis_notdisc

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.190217  0.086859       1        1  38.713623
4     -0.128546 -0.120595       2        1  27.495256
2     -0.156457  0.200245       3        1  17.465211
0      0.088870 -0.325000       4        1  10.427022
3      0.386350  0.158490       5        1   5.898887, topic_info=            Term          Freq         Total Category  logprob  loglift
0         accuse  27177.000000  27177.000000  Default  30.0000  30.0000
3    complainant   6958.000000   6958.000000  Default  29.0000  29.0000
4          enter  10126.000000  10126.000000  Default  28.0000  28.0000
6           find  10043.000000  10043.000000  Default  27.0000  27.0000
62         state   6958.000000   6958.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
376      several    381.775023   1263.476546   Topic5  -4.3192   1.6336
86       unknown    474.083602   2277.021526   Topic5  -4.1027   1.2612
155          son    374.388417   1306.931865   Topic5  -4.3388   1.5803
427     property    350.258097   1173.742989   Topic5  -4.4054   1.6211
39          hour    237.261597    904.590984   Topic5  -4.7949   1.4921

[252 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
240       4  0.999704  accident
0         1  0.868183    accuse
0         3  0.130218    accuse
0         4  0.001582    accuse
596       1  0.997652    action
...     ...       ...       ...
141       2  0.998927     white
30        2  0.998714     would
331       3  0.997586     write
156       5  0.998833      year
625       5  0.996614      yell

[271 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 1, 4])